# Initialization

In [ ]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

import numpy as np
import torch
import torch.nn as nn
import torchvision
from pkg_resources import packaging
import matplotlib.pyplot as plt
import clip
from PIL import Image
import os
from glob import glob

model, preprocess = clip.load("RN50x4")
print("Input resolution:", model.visual.input_resolution)

# Which neurons activate?

In [ ]:
class SaveActivations:
    def __init__(self):
        self.activations = None

    def __call__(self, module, input, output):
        self.activations = output.clone()

# Create an instance of the hook
hook = SaveActivations()

In [ ]:
class ModifiedResNetTraverser:
    def __init__(self, model):
        self.model = model
        self.target_layer_name = None
        self.target_layer = None

    def find_layer_by_name(self, target_layer_name):
        self.target_layer_name = target_layer_name
        self.target_layer = None
        self._traverse(self.model, target_layer_name.split('.'))
        return self.target_layer

    def _traverse(self, module, layer_names):
        if not layer_names:
            # If there are no more layer names, we've reached the target layer
            self.target_layer = module
            return

        current_layer_name = layer_names[0]
        remaining_layer_names = layer_names[1:]

        for name, sub_module in module.named_children():
            if name == current_layer_name:
                # Continue traversing with the next layer name
                self._traverse(sub_module, remaining_layer_names)

# Example usage
traverser = ModifiedResNetTraverser(model)
target_layer_name = "visual.layer4.5.conv3"
last_conv_layer = traverser.find_layer_by_name(target_layer_name)

In [ ]:
hook_handle = last_conv_layer.register_forward_hook(hook)

In [ ]:
from PIL import Image

original_images = []
images = []

for filename in [filename for filename in os.listdir("/content/trump1") if filename.endswith(".png") or filename.endswith(".jpg")]:
    image = Image.open(os.path.join("/content/trump1", filename)).convert("RGB")

    original_images.append(image)
    images.append(preprocess(image))

for filename in [filename for filename in os.listdir("/content/trump2") if filename.endswith(".png") or filename.endswith(".jpg")]:
    image = Image.open(os.path.join("/content/trump2", filename)).convert("RGB")

    original_images.append(image)
    images.append(preprocess(image))

In [ ]:
image_input = torch.tensor(np.stack(images)).cuda()
image_features = model.encode_image(image_input)

In [ ]:
activations = hook.activations
activations.size()

In [ ]:
max_values, _ = torch.max(activations.view(6, 2560, -1), dim=-1, keepdim=True)
max_values.size()

In [ ]:
import torch

# Squeeze the tensor to remove the singleton dimension (size 1)
input_tensor_squeezed = max_values.squeeze(dim=-1)

# Calculate the maximum value along the second dimension
max_values, indices = torch.topk(input_tensor_squeezed, k=10)

# Now 'max_values' has size [6, 5] and 'indices' has size [6, 5]
print(max_values.size())
print(indices)
print(max_values)

# Activation of neuron for different images

Test approach on one folder

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
trump_files = glob(os.path.join('/content/drive/MyDrive/patrec/trump', "**"))

In [ ]:
class SaveActivations:
    def __init__(self):
        self.activations = None

    def __call__(self, module, input, output):
        self.activations = output.clone()

# Create an instance of the hook
hook = SaveActivations()

In [ ]:
class ModifiedResNetTraverser:
    def __init__(self, model):
        self.model = model
        self.target_layer_name = None
        self.target_layer = None

    def find_layer_by_name(self, target_layer_name):
        self.target_layer_name = target_layer_name
        self.target_layer = None
        self._traverse(self.model, target_layer_name.split('.'))
        return self.target_layer

    def _traverse(self, module, layer_names):
        if not layer_names:
            # If there are no more layer names, we've reached the target layer
            self.target_layer = module
            return

        current_layer_name = layer_names[0]
        remaining_layer_names = layer_names[1:]

        for name, sub_module in module.named_children():
            if name == current_layer_name:
                # Continue traversing with the next layer name
                self._traverse(sub_module, remaining_layer_names)

# Example usage
traverser = ModifiedResNetTraverser(model)
target_layer_name = "visual.layer4.5.conv3"
last_conv_layer = traverser.find_layer_by_name(target_layer_name)

In [ ]:
hook_handle = last_conv_layer.register_forward_hook(hook)

In [ ]:
from PIL import Image

original_images = []
images = []

for filename in [filename for filename in os.listdir("/content/trump/profile") if filename.endswith(".png") or filename.endswith(".jpg")]:
    image = Image.open(os.path.join("/content/trump/profile", filename)).convert("RGB")

    original_images.append(image)
    images.append(preprocess(image))

image_input = torch.tensor(np.stack(images)).cuda()
image_features = model.encode_image(image_input)

activations = hook.activations
trump_activations = activations[:,89,:,:]

mean_values = torch.mean(trump_activations, dim=(1,2), keepdim=True)
values = mean_values[:,0,0].cpu().detach().numpy()
print(values)

# Activation of neuron for different image categories




Present results for Donald Trump and Mental Illness in pretty diagrams

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt

subfolder_names = ["profile", "art", "text", "partial", "politics", "non political", "rights", "music-games"]
data = {"Category": [], "Activation Values": []}

for subfolder in subfolder_names:
    folder_path = os.path.join("/content/drive/MyDrive/patrec/trump", subfolder)

    original_images = []
    images = []

    for filename in [filename for filename in os.listdir(folder_path) if filename.endswith(".png") or filename.endswith(".jpg")]:
        image = Image.open(os.path.join(folder_path, filename)).convert("RGB")

        original_images.append(image)
        images.append(preprocess(image))

    image_input = torch.tensor(np.stack(images)).cuda()
    image_features = model.encode_image(image_input)

    activations = hook.activations
    trump_activations = activations[:, 89, :, :]

    #mean:
    #mean_values = torch.mean(mental_activations, dim=(1, 2), keepdim=True)
    #values = mean_values[:, 0, 0].cpu().detach().numpy()
    #max:
    max_values, _ = torch.max(trump_activations.view(trump_activations.size(0), -1), dim=1)
    values = max_values.cpu().detach().numpy()
    data['Category'].extend([subfolder] * len(values))
    data['Activation Values'].extend(values)

df = pd.DataFrame(data)

# Create a displot
sns.displot(data=df, x="Activation Values", hue="Category", kind="kde", fill=True)

# Add labels and title
plt.xlabel('Activation Values')
plt.ylabel('# samples')
plt.title('Related to Donald Trump')
plt.xlim((min(df['Activation Values']),max(df['Activation Values'])))
plt.show()

In [ ]:
# Create a displot
sns.displot(data=df, x="Activation Values", hue="Category", kind="kde", fill=True, multiple="fill", palette='Set3', height=4, aspect=3)

# Add labels and title
plt.xlabel('Activation Values')
plt.ylabel('Conditional Probability')
plt.title('Related to Donald Trump')
plt.xlim((min(df['Activation Values']),max(df['Activation Values'])))
plt.show()

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt

subfolder_names = ["depression", "anxiety", "bad feeling", "psychology", "drugs", "unrelated", "travel-food-pet", "music-sports"]
data = {"Category": [], "Activation Values": []}

for subfolder in subfolder_names:
    folder_path = os.path.join("/content/drive/MyDrive/patrec/mental illness", subfolder)

    original_images = []
    images = []

    for filename in [filename for filename in os.listdir(folder_path) if filename.endswith(".png") or filename.endswith(".jpg")]:
        image = Image.open(os.path.join(folder_path, filename)).convert("RGB")

        original_images.append(image)
        images.append(preprocess(image))

    image_input = torch.tensor(np.stack(images)).cuda()
    image_features = model.encode_image(image_input)

    activations = hook.activations
    mental_activations = activations[:, 2191, :, :]

    #mean:
    #mean_values = torch.mean(mental_activations, dim=(1, 2), keepdim=True)
    #values = mean_values[:, 0, 0].cpu().detach().numpy()
    #max:
    max_values, _ = torch.max(mental_activations.view(mental_activations.size(0), -1), dim=1)
    values = max_values.cpu().detach().numpy()
    data['Category'].extend([subfolder] * len(values))
    data['Activation Values'].extend(values)

df = pd.DataFrame(data)

# Create a displot
sns.displot(data=df, x="Activation Values", hue="Category", kind="kde", fill=True)

# Add labels and title
plt.xlabel('Activation Values')
plt.ylabel('# samples')
plt.title('Related to Mental Illness')
plt.xlim((min(df['Activation Values']),max(df['Activation Values'])))
plt.show()

In [ ]:
# Create a displot
sns.displot(data=df, x="Activation Values", hue="Category", kind="kde", fill=True, multiple="fill", palette='Set3', height=4, aspect=3)

# Add labels and title
plt.xlabel('Activation Values')
plt.ylabel('Conditional Probability')
plt.title('Related to Mental Illness')
plt.xlim((min(df['Activation Values']),max(df['Activation Values'])))
plt.show()